# Comparing Altitude Estimates

We have 4 independent estimates of the altitude of the rocket.

 - Integration of the primary IMU (ADIS)
 - Pressure altimeter on the TeleMetrum altimeter
 - Pressure Altimeter on the ARTS2 altimeter
 - COTS GPS fixes right around apogee

The pressure altimeters both agree very well, but are both wrong. The GPS is probably the most correct after if finishes converging on a solution around MET+31 seconds. This might be because that neither have been calibrated for that days weather. The IMU altitude agrees very well with GPS.

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, argmax, add, average
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')


# ADIS IMU
columns = loadtxt("uncalibrated_integrated_altitude.csv", delimiter=',', unpack=True)
alt_time = columns[0]
imualt   = columns[1]


# TeleMetrum
columns = loadtxt("../TeleMetrum/2015-07-19-serial-1372-flight-0002.csv",
                  delimiter=',', comments='#',unpack=True,
                  usecols=(4,5,6,10,11,13,16,21,22,25))

tele_time = columns[0]
clock = columns[1]
rssi = columns[2]
tele_accel = columns[3]
tele_pressure = columns[4]
tele_alt = columns[5]
tele_temp = columns[6]
tele_gpslock, tele_gpsnsat = columns[7:9]
tele_gpsalt = columns[9]

FEET2METERS = 0.3048

# ARTS2
columns = loadtxt("../ARTS2/2015-07-19_ARTS2_Flight_Data_Interpreted_Data.csv",
                  delimiter=',', comments='#',unpack=True)
arts_time = columns[0]
arts_accel = columns[1]
arts_vel = columns[2]
arts_ialt = columns[3]
arts_palt = multiply(columns[4], FEET2METERS)


# VENUS
t_0 = 117853569585227

columns = loadtxt("../fc-data/V8A8.csv", delimiter=',', unpack=True)

seqn = columns[0]
venus_time = columns[1]
fix_mode = columns[2]
num_sv = columns[3]
TOW = columns[5]
latitude = columns[6]
longitude = columns[7]
altmsl = columns[9]
GDOP = columns[10]
PDOP = columns[1]
HDOP = columns[13]
VDOP = columns[14]
TDOP = columns[15]
vel_x = columns[18]
vel_y = columns[19]
vel_z = columns[20]

venus_time = subtract(venus_time, t_0)
venus_time = divide(venus_time, 1e9)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Altitude Comparisons")
plt.ylabel(r"Altitude [km]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(alt_time, imualt, alpha=0.75,     label="ADIS Integrated IMU")
plt.plot(tele_time, tele_alt, alpha=0.75,  label="TeleMetrum Pressure Alt")
plt.plot(arts_time, arts_palt, alpha=0.75, label="ARTS2 Pressure Alt")
plt.plot(venus_time, subtract(altmsl, altmsl[0]), 'k-', alpha=0.75,   label="Venus8 COTS GPS")

plt.xlim([-5,42])
#plt.ylim([-50,130])
ax1.legend(loc=4)
plt.show()